# Updating the names of rooms in a model

There are many situations one would like to update information in an IFC file. One case might be the need to update information on rooms in a model. 

The Rooms of a ifc model is of type [IfcSpace](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/FINAL/HTML/ifcproductextension/lexical/ifcspace.htm) and there are many posible ways of adding information to it both as direct attributes and in predefined or otherwise defined property sets. 

As IfcSpace inherits from [IfcRoot](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/TC1/HTML/ifckernel/lexical/ifcroot.htm) it has direct attributes of Name, Description, GlobalID and as it is aslo a [IfcSpatialStructureElement](https://standards.buildingsmart.org/IFC/RELEASE/IFC2x3/FINAL/HTML/ifcproductextension/lexical/ifcspatialstructureelement.htm) it also has the attribute of LongName. 

In this case we are interested in getting the Name, LongName, and GlobalID of all rooms in the model in an editable format to check and update Room Names. 

So, we'll walk through 

1. How to get an ifc file, query its room and direct attributes of Name, LongName and GlobalID
2. Write that to a csv file to handle in a 3rd party editor like eg. Google Spreadsheet, or Excel. 
3. Next we show how to read in this information from a csv file (separated by ",") and update the IFC model. 

## 1. Getting the rooms and attributes from IFC

In [1]:
# parse in the file using ifcopenshell
import ifcopenshell 

file = ifcopenshell.open("Grethes-hus-bok-2.ifc")

In [2]:
# get a reference to all objects in file that are of type "IfcSpace", effectively getting the list of all rooms in model. 
rooms = file.by_type("IfcSpace")

print(f"There are {len(rooms)} rooms in this model")

There are 8 rooms in this model


In [5]:
## Loop through the rooms and print the value of its attributes for mid-step visualization purposes.

for room in rooms:                                        
    print(f"""
    Room Name:\t\t {room.Name}
    LongName:\t\t {room.LongName}
    Room Description: {room.Description}\t\t
    Room GUID: {room.GlobalId}""")


    Room Name:		 1
    LongName:		 Kjøkken øst
    Room Description: None		
    Room GUID: 1gY9E27$nCk8ELruXgAK4X

    Room Name:		 2
    LongName:		 Stue øst
    Room Description: None		
    Room GUID: 1gY9E27$nCk8ELruXgAK4k

    Room Name:		 3
    LongName:		 Bod øst
    Room Description: None		
    Room GUID: 1gY9E27$nCk8ELruXgAK4h

    Room Name:		 4
    LongName:		 Bad øst
    Room Description: None		
    Room GUID: 1gY9E27$nCk8ELruXgAK4e

    Room Name:		 5
    LongName:		 Bad vest
    Room Description: None		
    Room GUID: 1gY9E27$nCk8ELruXgAK3L

    Room Name:		 6
    LongName:		 Bod vest
    Room Description: None		
    Room GUID: 1gY9E27$nCk8ELruXgAK3I

    Room Name:		 7
    LongName:		 Stue vest
    Room Description: None		
    Room GUID: 1gY9E27$nCk8ELruXgAK3V

    Room Name:		 8
    LongName:		 Kjøkken vest
    Room Description: None		
    Room GUID: 1gY9E27$nCk8ELruXgAK3S


## 2. Write out the GlobalID, Name and LongName attributes of all rooms into csv

In [6]:
## Write GUID, NAME, LongName of rooms to a csv file for use in other applications

import csv

with open('rooms.csv', 'w') as csvfile:
    fieldnames = ['GUID', 'Name','LongName','NameToUpdate']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for room in rooms:
        writer.writerow({'GUID': room.GlobalId, 'Name': room.Name, 'LongName': room.LongName,'NameToUpdate':''})


Documentation » The Python Standard Library
    
More: [csv — CSV File Reading and Writing](https://docs.python.org/3/library/csv.html)

## 3. Read in the content of a csv file to update the Name and LongName of the Rooms of the Given GlobalIDs

In [11]:
## Create a list to hold each room objects that later is to be added to model
## This assumes similar format as the csv that was written out. 
# FYI: This example file is edited and exported to csv again ("," delimetered) using Google Spreadsheet
# Other applications could export with other formatting and/or encoding

list_of_new_rooms = []

with open('rooms.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        list_of_new_rooms.append(row)
        print(row['GUID'], row['Name'],row['LongName']) ## uncomment this to print out each row. 

1gY9E27$nCk8ELruXgAK4X 1 Kjøkken øst
1gY9E27$nCk8ELruXgAK4k 2 Stue øst
1gY9E27$nCk8ELruXgAK4h 3 Bod øst
1gY9E27$nCk8ELruXgAK4e 4 Bad øst
1gY9E27$nCk8ELruXgAK3L 5 Bad vest
1gY9E27$nCk8ELruXgAK3I 6 Bod vest
1gY9E27$nCk8ELruXgAK3V 7 Stue vest
1gY9E27$nCk8ELruXgAK3S 8 Kjøkken vest


In [12]:
## Use file.by_guid("GlobalID") to get the exact objects from file, and update its attributes from file. 
## This gets the room by its guid from the file, then updates its Name and LongName attributes 

for room in list_of_new_rooms:
    room_by_guid = file.by_guid(room["GUID"])
    room_by_guid.Name = room['Name']
    room_by_guid.LongName = room['LongName']

In [13]:
## Now the Name and LongName attributes of the spaces have been updated. Then you can print a new version of the ifc file. 
## Now the file is stored in your home folder. 

new_file_name = "new_kontorbyggDIBK.ifc"
file.write(new_file_name)

# Extra - check out IfcCSV!

[BlenderBIM IFCCSV](https://wiki.osarch.org/index.php?title=BlenderBIM_IFCCSV) - The IFC CSV feature of IfcOpenShell is implemented in BlenderBIM Add-on

BlenderBIM tour with Dion Moult - IFCCSV - [video](https://www.youtube.com/watch?v=ZeMkG6jTzAg)